In [1]:
import xarray 
import os
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import imdlib as imd
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import scipy.io
import matplotlib.image as mpimg
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [2]:
x_train=pd.read_csv(r"C:\Users\ratho\Desktop\Software Group Project - III\Climate Data Modelling\Our Work\CSV Excel Data\SSTA Category 1901-2010.csv",index_col=0)
y_train=pd.read_csv(r"C:\Users\ratho\Desktop\Software Group Project - III\Climate Data Modelling\Our Work\Codes\Main Codes\SST Clusters\SSTA 8 Cluster Number 1901-2010.csv",index_col=0)
x_test=pd.read_csv(r"C:\Users\ratho\Desktop\Software Group Project - III\Climate Data Modelling\Our Work\CSV Excel Data\SSTA Category 2011-2022.csv",index_col=0)
y_test=pd.read_csv(r"C:\Users\ratho\Desktop\Software Group Project - III\Climate Data Modelling\Our Work\Codes\Main Codes\SST Clusters\SSTA 8 Cluster Number 2011-2022.csv",index_col=0)

In [3]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


In [4]:
model_params = {
    'svm': {
        'model': svm.SVC(decision_function_shape='ovo', gamma='auto'),
        'params': {
            'C': [1, 10, 20],
            'kernel': ['rbf', 'linear'],
            'degree': [2, 3, 4],
            'coef0': [0.0, 0.1, 0.5],
            'probability': [True, False],
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [1, 5, 10, 20, 50],
            'criterion': ['gini', 'entropy'],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
        }
    },
    'logistic_regression': {
        'model': LogisticRegression(solver='liblinear', multi_class='auto'),
        'params': {
            'C': [1, 5, 10],
            'penalty': ['l1', 'l2'],
        }
    },
    'naive_bayes_gaussian': {
        'model': GaussianNB(),
        'params': {}
    },
    'naive_bayes_multinomial': {
        'model': MultinomialNB(),
        'params': {}
    },
    'decision_tree': {
        'model': DecisionTreeClassifier(random_state=42),
        'params': {
            'criterion': [None,'gini', 'entropy'],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [None,2, 5, 10],
            'min_samples_leaf': [None,1, 2, 4],
        }
    },
    'linear_regression': {
        'model': LinearRegression(),
        'params': {}
    },
    'linear_discriminant_analysis': {
        'model': LinearDiscriminantAnalysis(),
        'params': {}
    },
}


In [5]:
y_train_array=y_train.values.ravel()
y_train_array

array([3, 0, 0, 7, 0, 0, 3, 7, 7, 7, 7, 0, 7, 0, 4, 3, 7, 7, 4, 5, 3, 0,
       0, 0, 3, 5, 3, 0, 7, 0, 4, 0, 3, 3, 3, 5, 2, 3, 3, 4, 1, 4, 3, 5,
       2, 4, 5, 3, 3, 3, 3, 4, 4, 4, 3, 3, 4, 4, 5, 4, 5, 5, 5, 2, 3, 4,
       5, 3, 1, 1, 5, 5, 2, 5, 3, 3, 5, 4, 1, 1, 4, 1, 1, 2, 2, 2, 1, 6,
       2, 6, 6, 1, 1, 6, 6, 2, 1, 1, 2, 2, 6, 6, 6, 6, 6, 6, 6, 2, 6, 6],
      dtype=int64)

In [6]:
from sklearn.model_selection import GridSearchCV
import pandas as pd
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(x_train, y_train_array)
    y_pred = clf.predict(x_test)
    y_pred = np.ceil(y_pred)
    y_test_array= y_test.values.ravel()
    y_test_array
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'accuracy_score': accuracy_score(y_pred,y_test_array),
        'best_params': clf.best_params_,
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','accuracy_score','best_params'])
df

C:\Users\ratho\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:976: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan 0.76363636 0.76363636 0.73636364
        nan 0.7        0.70909091 0.71818182        n

,model,best_score,accuracy_score,best_params
0,svm,0.900000,1.000000,"{'C': 1, 'coef0': 0.0, 'degree': 2, 'kernel': ..."
1,random_forest,0.890909,0.833333,"{'criterion': 'entropy', 'max_depth': 30, 'min..."
2,logistic_regression,0.718182,0.833333,"{'C': 5, 'penalty': 'l1'}"
3,naive_bayes_gaussian,0.900000,0.916667,{}
4,naive_bayes_multinomial,0.654545,0.916667,{}
5,decision_tree,0.827273,0.750000,"{'criterion': 'entropy', 'max_depth': None, 'm..."
6,linear_regression,-0.855291,0.000000,{}
7,linear_discriminant_analysis,0.809091,0.833333,{}
